# Sales Copilot

Additional concepts:

- Copilot determines intent. Either goes to load balance endpoint or Assistant depending on user intent.
- Thread management for multiple users
  - Using shelves to store the user threads
  - Thread will be created or reused as different users access the assistant
- Relationships in files

## Diagram

- **Note**: Not implemented, promptflow and weather services

![Architecture Diagram](../images/copilot-with-assistant.png)

### Load the required libraries

In [1]:
import os
import sales_agent as agent
from pathlib import Path

from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types import FileObject


### Load the environment variables

In [2]:
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

### Create the AzureOpenAI clients

In [3]:
load_balanced_client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

In [4]:
sales = agent.get_sales_agent()

data/sales/sellers.csv
data/sales/customers.csv
data/sales/orders.csv
data/sales/inventory.csv


In [5]:
def semantic_intent(prompt:str)->str:
    prompt_template = """system:
You are an agent that can determine intent from the following list of intents and return the intent that best matches the user's question.
 
List of intents:
SalesAgent: questions or operations related to calculations, customers, sales, orders, inventory, items, and sellers.
OtherAgent: any other question

user:
<QUESTION>

Output in ONE word."""


    full_prompt = prompt_template.replace("<QUESTION>", prompt)
    completion = load_balanced_client.chat.completions.create(
            model=api_deployment_name,
            messages=[
                {
                    "role": "user",
                    "content": full_prompt,
                },
            ],
            max_tokens=2,
            temperature=0.1           
        )    
    try:
        return completion.choices[0].message.content
    except:
        return "Unknown"
    

### Go to load balance endpoint or to the Assistant based on intent

In [6]:
def process_for_intent(user: str, user_id: str, prompt: str):
    intent = semantic_intent(prompt)
    print(f"Intent: {intent}")
    
    if intent=="SalesAgent":
        sales.process_prompt(user, user_id, "What customers are in Florida?")
    elif intent=="OtherAgent":
        completion = load_balanced_client.chat.completions.create(
            model=api_deployment_name,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ]
        )
        print(completion.choices[0].message.content)
        
    else:        
        print("Unknown Intent")

### Have a conversation with the Assistant

In [7]:
process_for_intent("Suzan", "user_123", "What is the speed of light?")

Intent: OtherAgent
The speed of light in a vacuum is a fundamental constant of nature and is one of the most accurately measured quantities in science. It is approximately 299,792,458 meters per second, or about 186,282 miles per second. This value is often represented by the symbol "c" in equations, such as Einstein's famous equation E=mc², where E is energy, m is mass, and c is the speed of light in a vacuum.


In [8]:
process_for_intent("John", "user_123", "What customers are in Florida?")

Intent: SalesAgent
processing ...
user: John:
What customers are in Florida?

assistant:
To assist you in identifying customers located in Florida, I'll need to examine the contents of the files you uploaded. Let's start by inspecting these files to determine whether they contain customer data and if so, whether the data includes location information such as the state. I will then filter the information to identify customers in Florida. Please hold on while I process the files.

assistant:
It appears that the data frames generated from the files do not contain a column labeled 'State' or the files provided might not have been in a readable spreadsheet format. To continue assisting you, I'll try a different approach by inspecting the raw content of the files to search for location information that might indicate customers in Florida. Please hold on while I process the files again.

assistant:
The analysis indicates that the name "John Doe" is associated with a location information patte

In [ ]:
process_for_intent("Mary", "user_234", "What seller has had the most sales?")

In [ ]:
process_for_intent("John", "user_123", "What is the most sold product?")

In [ ]:
process_for_intent("Mary", "user_234", "Chart product sales by State.")

### Cleanup

In [9]:
sales.cleanup()

AssistantDeleted(id='asst_BfLpf9dagtmvf2JEarUylA7J', deleted=True, object='assistant.deleted')
Deleting:  0  threads.
Deleting:  4  files.
FileDeleted(id='assistant-2vfOqXosQOllazKZJ6C4DFvs', deleted=True, object='file')
FileDeleted(id='assistant-qJ5sVkZxZvmG4a0A0ImmFW6z', deleted=True, object='file')
FileDeleted(id='assistant-h1d5jFTO2xnuqAol3RquMRbh', deleted=True, object='file')
FileDeleted(id='assistant-xNoSaC2JQqK0iEkcB99lB4Ev', deleted=True, object='file')
